# EPyR Tools - Getting Started

Welcome to EPyR Tools! This notebook demonstrates how to load and visualize EPR data from Bruker spectrometers.

## What you'll learn:
- Loading BES3T (.dsc/.dta) and ESP (.par/.spc) files
- Handling 1D and 2D EPR data 
- Basic visualization and parameter extraction
- Simple data export

In [ ]:
# Import required libraries
import sys
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Add EPyR Tools to Python path (for notebook usage)
# This ensures the notebook can find epyr whether installed or not
current_dir = Path().resolve()
epyr_root = current_dir.parent.parent  # Go up two levels from notebooks to project root

# Add both the project root and epyr package to path
if str(epyr_root) not in sys.path:
    sys.path.insert(0, str(epyr_root))
    
print(f\"Looking for EPyR Tools in: {epyr_root}\")\nprint(f\"EPyR package should be at: {epyr_root / 'epyr'}\")\n\n# Check if epyr directory exists\nepyr_package_dir = epyr_root / 'epyr'\nif epyr_package_dir.exists():\n    print(f\"✅ EPyR package directory found\")\nelse:\n    print(f\"❌ EPyR package directory not found\")\n    print(f\"Please make sure you're running this notebook from examples/notebooks/\")\n\n# Import EPyR Tools\ntry:\n    import epyr\n    print(\"\\n✅ EPyR Tools loaded successfully!\")\n    print(f\"Version: {epyr.__version__}\")\nexcept ImportError as e:\n    print(f\"\\n❌ Failed to import EPyR Tools: {e}\")\n    print(\"\\nTroubleshooting:\")\n    print(\"1. Make sure you're in the examples/notebooks/ directory\")\n    print(\"2. Try: pip install -e . (from project root)\")\n    print(\"3. Check that epyr/ directory exists in project root\")"

## 1. Find Available Data Files

In [ ]:
# Look for sample data files in the consolidated data directory
data_dir = Path(\"../data\")

print(f\"Looking for data files in: {data_dir.resolve()}\")\n\n# Find all EPR files in the single data directory\nbes3t_files = list(data_dir.glob(\"*.DSC\")) + list(data_dir.glob(\"*.dsc\"))\nesp_files = list(data_dir.glob(\"*.par\"))\n\nprint(f\"\\nFound {len(bes3t_files)} BES3T files and {len(esp_files)} ESP files\")\n\nprint(\"\\nAvailable files:\")\nall_files = [(f, \"BES3T\") for f in bes3t_files] + [(f, \"ESP\") for f in esp_files]\n\nfor i, (file_path, file_type) in enumerate(all_files):\n    file_size = file_path.stat().st_size / 1024  # KB\n    print(f\"{i+1}. [{file_type}] {file_path.name} ({file_size:.1f} KB)\")\n    \nif not all_files:\n    print(\"❌ No EPR files found!\")\n    print(f\"Please add your .DSC/.DTA or .PAR/.SPC files to: {data_dir.resolve()}\")"

## 2. Load a 1D EPR Spectrum (BES3T format)

In [ ]:
# Load the 1D BES3T file
file_1d = data_dir / \"130406SB_CaWO4_Er_CW_5K_20.DSC\"\n\nif file_1d.exists():\n    print(f\"Loading 1D file: {file_1d.name}\")\n    \n    # Load the data\n    x1, y1, params1, filepath1 = epyr.eprload(str(file_1d), plot_if_possible=False)\n    \n    if x1 is not None and y1 is not None:\n        print(f\"✅ Loaded successfully!\")\n        print(f\"Data points: {len(x1)}\")\n        print(f\"Field range: {x1.min():.1f} - {x1.max():.1f} G\")\n        print(f\"Signal range: {y1.min():.2e} - {y1.max():.2e}\")\n        \n        # Show some key parameters\n        print(f\"\\nKey parameters:\")\n        key_params = ['MWFQ', 'MWPW', 'HCF', 'HSW', 'RES', 'AVGS']\n        for param in key_params:\n            if param in params1:\n                print(f\"  {param}: {params1[param]}\")\n    else:\n        print(\"❌ Failed to load data from file\")\nelse:\n    print(\"❌ 1D file not found\")\n    print(f\"Expected file: {file_1d}\")\n    print(\"Available files:\", [f.name for f in bes3t_files])"

In [ ]:
# Plot the 1D spectrum
if 'x1' in locals():
    plt.figure(figsize=(12, 6))
    plt.plot(x1, y1, 'b-', linewidth=1.5)
    plt.xlabel('Magnetic Field (G)')
    plt.ylabel('EPR Signal (a.u.)')
    plt.title(f'1D EPR Spectrum: {file_1d.stem}')
    plt.grid(True, alpha=0.3)
    
    # Add info box
    info_text = f"Points: {len(x1)}\nRange: {x1.max()-x1.min():.0f} G"
    plt.text(0.02, 0.98, info_text, transform=plt.gca().transAxes,
             verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    plt.tight_layout()
    plt.show()
    
    print(f"📊 1D spectrum plotted successfully!")

## 3. Load a 2D EPR Dataset (BES3T format)

In [ ]:
# Load the 2D BES3T file (Rabi experiment)\nfile_2d = data_dir / \"Rabi2D_GdCaWO4_13dB_3057G.DSC\"\n\nif file_2d.exists():\n    print(f\"Loading 2D file: {file_2d.name}\")\n    \n    # Load the data\n    x2_axes, y2, params2, filepath2 = epyr.eprload(str(file_2d), plot_if_possible=False)\n    \n    if x2_axes is not None and y2 is not None:\n        print(f\"✅ Loaded successfully!\")\n        print(f\"Data type: x_axes is {type(x2_axes)} with {len(x2_axes)} axes\")\n        print(f\"Data shape: {y2.shape}\")\n        print(f\"This is a {y2.shape[0]} x {y2.shape[1]} 2D dataset\")\n        \n        # Extract axes - x2_axes is a list of [x_axis, y_axis]\n        if len(x2_axes) >= 2:\n            x_axis = x2_axes[0]  # First dimension (e.g., field)\n            y_axis = x2_axes[1]  # Second dimension (e.g., time, power, etc.)\n            \n            print(f\"X-axis (dim 1): {len(x_axis) if hasattr(x_axis, '__len__') else 'scalar'} points\")\n            print(f\"Y-axis (dim 2): {len(y_axis) if hasattr(y_axis, '__len__') else 'scalar'} points\")\n            \n            # Show some key parameters\n            print(f\"\\nKey parameters:\")\n            key_params = ['MWFQ', 'MWPW', 'HCF', 'HSW']\n            for param in key_params:\n                if param in params2:\n                    print(f\"  {param}: {params2[param]}\")\n    else:\n        print(\"❌ Failed to load data from file\")\nelse:\n    print(\"❌ 2D file not found\")\n    print(f\"Expected file: {file_2d}\")\n    print(\"Available files:\", [f.name for f in bes3t_files])"

In [ ]:
# Plot the 2D data
if 'x2_axes' in locals() and len(x2_axes) >= 2:
    
    # Handle complex data by taking magnitude
    if np.iscomplexobj(y2):
        y2_plot = np.abs(y2)  # Use magnitude for complex data
        data_info = "(Complex data - showing magnitude)"
    else:
        y2_plot = y2
        data_info = "(Real data)"
    
    # Create figure with subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Plot 1: 2D color map
    im = ax1.imshow(y2_plot, aspect='auto', origin='lower', cmap='viridis')
    ax1.set_xlabel('Field Points')
    ax1.set_ylabel('Time/Parameter Points')
    ax1.set_title(f'2D EPR Data {data_info}')
    plt.colorbar(im, ax=ax1, label='Signal (a.u.)')
    
    # Plot 2: Few representative slices
    n_slices = min(5, y2_plot.shape[0])
    for i in range(n_slices):
        if hasattr(x_axis, '__len__'):
            ax2.plot(x_axis, y2_plot[i, :], alpha=0.7, label=f'Slice {i+1}')
        else:
            ax2.plot(y2_plot[i, :], alpha=0.7, label=f'Slice {i+1}')
    
    ax2.set_xlabel('Field (G)' if hasattr(x_axis, '__len__') else 'Points')
    ax2.set_ylabel('EPR Signal (a.u.)')
    ax2.set_title('Representative Spectra')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"📊 2D dataset plotted successfully!")
    print(f"Data type: {'Complex' if np.iscomplexobj(y2) else 'Real'}")
    print(f"Showing {n_slices} representative slices from {y2_plot.shape[0]} total")

## 4. Simple Data Analysis

In [ ]:
# Basic analysis on 1D data
if 'x1' in locals():
    print("📈 1D Spectrum Analysis:")
    print(f"  Mean signal: {np.mean(y1):.2e}")
    print(f"  Signal std: {np.std(y1):.2e}")
    print(f"  Peak-to-peak: {np.ptp(y1):.2e}")
    print(f"  Signal-to-noise ratio: {np.ptp(y1)/np.std(y1):.1f}")
    
    # Find approximate center
    center_idx = np.argmax(np.abs(y1))
    center_field = x1[center_idx]
    print(f"  Strongest signal at: {center_field:.1f} G")

# Basic analysis on 2D data  
if 'y2' in locals():
    print(f"\\n📊 2D Dataset Analysis:")
    print(f"  Data shape: {y2.shape[0]} spectra x {y2.shape[1]} points")
    print(f"  Total data points: {y2.size:,}")
    print(f"  Data type: {'Complex' if np.iscomplexobj(y2) else 'Real'}")
    
    # Use magnitude for complex data in analysis
    y2_analysis = np.abs(y2) if np.iscomplexobj(y2) else y2
    print(f"  Overall signal range: {y2_analysis.min():.2e} to {y2_analysis.max():.2e}")
    print(f"  Mean signal: {np.mean(y2_analysis):.2e}")

## 5. Export Data to Common Formats

In [ ]:
# Create output directory
output_dir = Path("../data/processed")
output_dir.mkdir(exist_ok=True)

# Export 1D data
if 'x1' in locals():
    print("💾 Exporting 1D data...")
    
    # Export to CSV
    csv_file = output_dir / "spectrum_1D.csv"
    with open(csv_file, 'w') as f:
        f.write("# EPyR Tools Export - 1D EPR Spectrum\\n")
        f.write(f"# File: {file_1d.name}\\n")
        f.write(f"# Points: {len(x1)}\\n")
        f.write("field_gauss,intensity_au\\n")
        for xi, yi in zip(x1, y1):
            f.write(f"{xi:.4f},{yi:.6e}\\n")
    print(f"  ✅ CSV saved: {csv_file.name}")
    
    # Export to simple text
    txt_file = output_dir / "spectrum_1D.txt"
    np.savetxt(txt_file, np.column_stack([x1, y1]), 
               header="Field(G) Intensity(a.u.)", fmt="%.4f %.6e")
    print(f"  ✅ TXT saved: {txt_file.name}")

# Export 2D data summary
if 'y2' in locals():
    print(f"\\n💾 Exporting 2D data summary...")
    
    # Use magnitude for complex data export
    y2_export = np.abs(y2) if np.iscomplexobj(y2) else y2
    
    # Export first few spectra
    for i in range(min(3, y2_export.shape[0])):
        csv_file = output_dir / f"spectrum_2D_slice_{i+1}.csv"
        with open(csv_file, 'w') as f:
            f.write(f"# EPyR Tools Export - 2D EPR Spectrum Slice {i+1}\\n")
            f.write(f"# File: {file_2d.name}\\n")
            f.write(f"# Data type: {'Complex (exported as magnitude)' if np.iscomplexobj(y2) else 'Real'}\\n")
            f.write(f"# Generated by EPyR Tools v0.1.2\\n")
            f.write("index,intensity_au\\n")
            for j, yi in enumerate(y2_export[i, :]):
                f.write(f"{j},{yi:.6e}\\n")
        print(f"  ✅ Slice {i+1} saved: {csv_file.name}")
    
    # Save full 2D data as numpy format for later use
    npz_file = output_dir / "spectrum_2D_full.npz"
    np.savez(npz_file, 
             data=y2_export, 
             original_complex=np.iscomplexobj(y2),
             shape=y2.shape)
    print(f"  ✅ Full 2D data saved: {npz_file.name}")

print(f"\\n📁 All exports saved to: {output_dir}")

## Summary

🎉 **Congratulations!** You've successfully:

✅ Loaded 1D EPR data (BES3T format)  
✅ Loaded 2D EPR data (multi-dimensional measurements)  
✅ Visualized both data types  
✅ Performed basic analysis  
✅ Exported data to common formats  

## Next Steps

- **Baseline Correction**: Remove drift and artifacts
- **Peak Analysis**: Find and fit EPR lines
- **Quantitative Analysis**: Calculate g-factors and concentrations
- **Batch Processing**: Handle multiple files automatically

## Need Help?

- Check the EPyR Tools documentation
- Look at example scripts in `examples/scripts/`
- Visit the GitHub repository for updates